# Der Höhepunkt: SQL, Spark-Tables, Reporting ...

* SQL Abfragen auf Dataframes anwenden
* Mit Spark Datenbanken & Tabellen arbeiten
* User-Defined Functions benutzen
* Datenmegen joinen
* Window Operationen
* Testdaten erzeugen, welche zwei Geschäftsprozesse aus der Versicherungswelt abbilden
* Die ersten (und wesentlichsten) Schritte für ein sinnvolles Monitoring dieser Prozesse gehen


## Wie immer eine Spark-Application registieren

In [ ]:
import findspark
findspark.init()
findspark.find()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
        .builder
        .appName("sql")
        .config("spark.dynamicAllocation.enabled",False)
        .config("spark.sql.adaptive.enabled",False)
        .master("local[4]")
        .getOrCreate()
)
spark

## Nun Testdaten erzeugen
Hierzu haben wir ein Beispiel aus der Versicherungswelt vorbereiten. Vielleicht ist es dem in der Signal sogar ein bisschen ähnlich ;-)


![](architecture-level-01.dio.svg)

In [ ]:
import generate_test_data
generate_test_data.main(1000)

Wir sehen nun auf im Dateibrowser eine Liste von Dateien, die überwiegend fachliche Events enthalten, die zwei Geschäftsprozesse abbilden:

### Ein Kunden bekommt einen neuen Vertrag

Die Systeme welche die Events schreiben liegen in der Verantwortung verschiedener Teams:

#### Team Antrag
 * **Antrag Erzeugt:** Der Kunde geht im Internet auf ein System um einen Antrag auf einen Versicherungsvertrag zu stellen. Dieses System validert schon ein wenig, versucht einen schon bestehenden Kunden im Partner-Systeme zu finden und schmeißt im Anschluss dieses Business-Event.
 * **AB-Test:** das Team Antrag ist stehst bemüht die User-Experience zu optimieren. So will es zum Beispiel die Durchlaufzeit auf ihrem eigenen System zu verkürzen. So variert es z.B. die Farbe des Knopfes. Wenn es bei einem Antrag von der Standardfarbe des Knopfes abweicht, schreibt es dieses technische Event
 * **Kunde hat Angebot aktzeptiert:** Nch
 * **Kunde hat Angebot abgelehnt:**
 
#### Team Vertrag
 * **Antrag abgelehnt**: Ein Vertrag kann vom Vertragssystem abgelehnt werden. Dieses fragt z.B. noch bei Fraud an und validert auch sonst den Antrag viel "besser" als es das Antragssystem könnte.
 * **Kunde angelegt**: (eigentlich müsste ein anderes System dieses Event schreiben, aber in unserem Beispiel soll das mal OK sein) Wenn in einem Antrag nicht schon eine Referenz auf einen Kunden vorhanden ist, dann legt das Vertragssystem einen Kunden an und informiert Gott und die Welt hierüber.
 * **Vertrag angeboten**: Wenn ein Vertragsantrag aus Sicht des Vertragssystem gut aussieht, dann gibt es den Vertrag frei zur Annahme durch den Kunden. 
 * **Vertrag policiert**: Wenn der Kunden ein Vertragsangebot akzeptiert hat, policieren wir den Vertrag. Nun ist der Kunde versichert. Jetzt können bedenkenlos Schadensfälle eintreten.
 
#### Team Schaden
 * **Schaden reguliert**: wenn ein Schadensfall geprüft wurde und alles passt, dann überweisen wir Geld

#### Übrigens
**kunden.csv** ist nur ein Container mit allen Kunden ;-)



## Erste Blicke in unsere Testdaten

In [ ]:
kunden_df = spark.read.option("header", True).option("inferSchema", True).csv("kunden.csv")
kunden_df.printSchema()
kunden_df.show(4, truncate=False)

Soweit so bekannt. Wir könnten jetzt auf die Daten zugreifen in dem wir wie in den vorhergegangen Kapiteln filter anwenden, Null-Werte entfernen usw.
Wir möchten aber absofort ganz normal mit SQL auf unsere Daten zugreifen. Dazu müssen wir zu einem Dataframe in Spark einen _View_ anlegen. Diese Views werden dann ganz normal wie Datenbanktabellen in SQL-Datenbanken behandelt (read-only).

In [ ]:
kunden_df.createOrReplaceTempView("kunden")

**Beachte:** die Funtion gibt nichts zurück. In der Spark-Application ist jetzt eine eine Tabelle unter dem Namen "kunden" verfügbar!

In [ ]:
spark.sql("SELECT * FROM kunden")

Wir sehen das wir ein Dataframe zurückbekommen. Auf diesem können wir dann wieder die bekannten Operationen ausführen. Beachte, ob Du die Dateframe-API oder die SQL-API nutzt, hat in der Regel **keinen** Impact auf die Performance!

**Übung** setzte hier mal ein paar SQLs gegen diese Tabelle ab. Filter nach bestimmten Namen... 

In [ ]:
spark.sql('SELECT * FROM kunden').show()

## Alle Testdaten importieren
und views anlegen

In [ ]:
# AB-test
ab_test_schema = "AntragsId STRING"
ab_test_df = spark.read.option("header", True).schema(ab_test_schema).csv("ab_test.csv")
ab_test_df.createOrReplaceTempView("ab_test")

# Antrag Abgelehnt
antrag_abgelehnt_schema = "AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP, Grund STRING"
antrag_abgelehnt_df = spark.read.option("header", True).schema(antrag_abgelehnt_schema).csv("antrag_abgelehnt.csv")
antrag_abgelehnt_df.createOrReplaceTempView("antrag_abgelehnt")

# Antrag Erzeugt
antrag_erzeugt_schema = "AntragsId STRING, StartZeit TIMESTAMP, EndZeit TIMESTAMP, KundenId STRING"
antrag_erzeugt_df = spark.read.option("header", True).schema(antrag_erzeugt_schema).csv("antrag_erzeugt.csv")
antrag_erzeugt_df.createOrReplaceTempView("antrag_erzeugt")

# Kunde Angelegt
kunde_angelegt_schema = "KundenId STRING, TimeStamp TIMESTAMP"
kunde_angelegt_df = spark.read.option("header", True).schema(kunde_angelegt_schema).csv("kunde_angelegt.csv")
kunde_angelegt_df.createOrReplaceTempView("kunde_angelegt")

# Kunde hat Angebot Abgelehnt
kunde_hat_angebot_abgelehnt_schema = "VertragsId STRING, AntragsId STRING, Grund STRING, TimeStamp TIMESTAMP"
kunde_hat_angebot_abgelehnt_df = spark.read.option("header", True).schema(kunde_hat_angebot_abgelehnt_schema).csv("kunde_hat_angebot_abgelehnt.csv")
kunde_hat_angebot_abgelehnt_df .createOrReplaceTempView("kunde_hat_angebot_abgelehnt")

# Kunde hat Angebot Akzeptiert
kunde_hat_angebot_akzeptiert_schema = "VertragsId STRING, AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP"
kunde_hat_angebot_akzeptiert_df = spark.read.option("header", True).schema(kunde_hat_angebot_akzeptiert_schema).csv("kunde_hat_angebot_akzeptiert.csv")
kunde_hat_angebot_akzeptiert_df.createOrReplaceTempView("kunde_hat_angebot_akzeptiert")

# Schaden Gemeldet
schaden_gemeldet_schema = "VertagsId STRING, SchadensId STRING, Schadenshoehe INT, TimeStamp TIMESTAMP"
schaden_gemeldet_df = spark.read.option("header", True).schema(schaden_gemeldet_schema).csv("schaden_gemeldet.csv")
schaden_gemeldet_df.createOrReplaceTempView("schaden_gemeldet")

# Schaden Reguliert
schaden_reguliert_schema = "SchadensId STRING, TimeStamp TIMESTAMP"
schaden_reguliert_df = spark.read.option("header", True).schema(schaden_reguliert_schema).csv("schaden_reguliert.csv")
schaden_reguliert_df.createOrReplaceTempView("schaden_reguliert")

# Vertrag Angeboten
vertrag_angeboten_schema = "VertragsId STRING, AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP"
vertrag_angeboten_df = spark.read.option("header", True).schema(vertrag_angeboten_schema).csv("vertrag_angeboten.csv")
vertrag_angeboten_df.createOrReplaceTempView("vertrag_angeboten")

# Vertrag Policiert
vertrag_policiert_schema = "VertragsId STRING, TimeStamp TIMESTAMP"
vertrag_policiert_df = spark.read.option("header", True).schema(vertrag_policiert_schema).csv("vertrag_policiert.csv")
vertrag_policiert_df.createOrReplaceTempView("vertrag_policiert")

In [ ]:
# alles da?
spark.sql("SHOW TABLES").show(truncate=False)

In [ ]:
# Stichprobe bzgl. der Datentypen?
spark.sql("DESCRIBE TABLE EXTENDED schaden_gemeldet").show()

## Ein erstes Reporting

In [ ]:
# Zeige alle Anträge an, bei den wir die Farbe des Knopfs verändert haben

# erstmal alle Anträge

spark.sql("SELECT count(1) FROM antrag_erzeugt").show()
spark.sql("""
     SELECT count(1)
       FROM antrag_erzeugt
 INNER JOIN ab_test
         ON antrag_erzeugt.AntragsId = ab_test.AntragsId
""").show()

## Spark Tables

Bisher haben wir gesehen wie wir CSV Dateien laden und speichern können.
Dabei mussten wir allerdings z.B. immer ein Schema angeben.
Das ist auf Dauer lästig und mindestens aufwendig.


![](spark-catalog.dio.svg)

In [ ]:
# beispiel mit dem Kunde angelegt Event
kunde_angelegt_df.printSchema()

In [ ]:
kunde_angelegt_df.write.mode("overwrite").saveAsTable("myTestTable")

In [ ]:
kunde_angelegt_df = spark.read.table("myTestTable")

In [ ]:
kunde_angelegt_df.printSchema()

wir können aber auch direkt mit SQL aus einer Table lesen

In [ ]:
spark.sql("SELECT * FROM myTestTable")

### Typen von Catalogs


* **In-memory Catalog** Wenn eine Spark-Sitzung ended, wird auch der Catalog aufgeräumt
* **Persistent Catalog** Metadaten werden permanent gespeichert, Spark hat einen [Hive](https://hive.apache.org/)-basierten Catalog eingebaut.

### Typen von Tables

* **Managed Tables** Spark kümmert sich um den Lebenszyklus. Es werden Daten und Schemata verwaltet. Nützlich für staging Daten. Wenn eine Tabelle gelöscht wird, dann werden Daten und Schemata gelöscht.
* **Unmanged Tables/External Tables** Das Schema wird auch hier von Spark gemanaged, aber die Daten in einer frei wählbaren location. Löschen einer Tabelle löscht hier nur das Schema. Nützlich für das Ergebniss von ETL Pipelines

In [ ]:
import findspark
findspark.init()

from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
        .builder
        .appName("SparkTablesApp")
        .master("local[4]")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.sql.adaptive.enabled", "false")
        
        .enableHiveSupport() # <---------------------------------
    
        .getOrCreate()
)

sc = spark.sparkContext
spark

### Nun eine Datenbank in Hive erstellen

In [ ]:
spark.sql("""
  SHOW DATABASES
  """).show()

In [ ]:
spark.sql("""
    CREATE DATABASE IF NOT EXISTS Foo
""").show()

In [ ]:
spark.sql("""
    SHOW DATABASES
""").show()

In [ ]:
kunde_angelegt_df.write.mode("overwrite").saveAsTable("foo.bar")
# wurde als managed table abgespeichert, da keine externe location angegeben wurde

In [ ]:
spark.sql("""
    SHOW TABLES in foo
""").show(truncate=False)

In [ ]:
spark.sql("""
SELECT * from foo.bar
""").show()

In [ ]:
# oder auch die tablle direkt in ein Dataframe lesen

tmp = spark.read.table("foo.bar")
tmp.printSchema()
tmp.show(2)

In [ ]:
# Nun schauen wir uns mal die details der Tabelle an
spark.sql("""
    DESCRIBE TABLE EXTENDED foo.bar
""").show(truncate=False)

In [ ]:
# Nun eine UNMANAGED Table anlegen
(
    kunden_df
        .write
        .mode("overwrite")
        .option("path", "/home/pupil/spark-course/course/03-SQL/spark-warehouse/persistent/kunden.parquet")
        #.option("format", "csv") # defaults to parquet
        .saveAsTable("foo.bar")
)

In [ ]:
spark.sql("""
 DESCRIBE TABLE EXTENDED foo.bar
""").show(truncate=False)

In [ ]:
spark.sql("""
DROP TABLE foo.bar
""").show(truncate=False)

im datei browser anschauen, dass die Datein nicht wirklich gelöscht wurden
Nun mit SQL die Tabelle wieder herstllen


In [ ]:
# Nun eine Table direkt aus parquet herstellen (parquet hat Schema eingebaut ;-) )
spark.sql("""
    CREATE TABLE  foo.bar
    USING PARQUET
    LOCATION "/home/pupil/spark-course/course/03-SQL/spark-warehouse/persistent/kunden.parquet"
""").show(truncate=False)

In [ ]:
spark.sql("""
    SELECT * FROM foo.bar
""").show()

**Take Away** Spark Tables machen die Entwicklung schneller und einfacher

## Spark User Defined Functions

## Joins

## Windows